In [1]:
!pip install gensim
!wget -nc https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz 
!pip install -U textblob

from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt

import os
import string
import pandas as pd
import numpy as np
import re
import pprint
from urllib.request import urlopen

import gensim
from gensim import corpora, models, similarities
import nltk
from nltk import word_tokenize, tokenize
nltk.download('punkt')

/bin/sh: wget: command not found
Requirement already up-to-date: textblob in /Users/aisetoyama/opt/anaconda3/lib/python3.7/site-packages (0.15.3)


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/aisetoyama/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
biden_df = pd.read_csv('hashtag_joebiden.csv', lineterminator='\n')
trump_df = pd.read_csv('hashtag_donaldtrump.csv', lineterminator='\n')


#converted tweet_created into a datetime object and made it into a new column
import datetime
def clean_date(df):
    datetime_obj = []
    for elem in df['created_at']:
        obj = datetime.datetime.strptime(elem, '%Y-%m-%d %H:%M:%S').date()
        datetime_obj.append(obj)
    df['datetime'] = datetime_obj
clean_date(biden_df)   
clean_date(trump_df)  

#selected rows that were before November 3 inclusive
biden_df = biden_df[biden_df['datetime'] <= datetime.date(2020, 11, 3)]

trump_df = trump_df[trump_df['datetime'] <= datetime.date(2020, 11, 3)]


In [3]:
# The sentiment property returns a namedtuple of the form Sentiment(polarity, subjectivity). 
# The polarity score is a float within the range [-1.0, 1.0]. 
# The subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.
from textblob import TextBlob
def textblob_polarity(tweets): 
    polarity = []
    for tweet in tweets:
        testimonial = TextBlob(tweet)
        polarity.append(testimonial.sentiment.polarity)
    return polarity

In [4]:
biden_df['textblob_polarity'] = textblob_polarity(biden_df['tweet'])
trump_df['textblob_polarity'] = textblob_polarity(trump_df['tweet'])

In [5]:

biden_df = biden_df[biden_df['textblob_polarity'] != 0.0]
biden_df = biden_df.reset_index()

trump_df = trump_df[trump_df['textblob_polarity'] != 0.0]
trump_df = trump_df.reset_index()

In [6]:
def forBiden(polarity):
    verdicts = []
    for score in polarity: 
        if score > 0:
            verdicts.append('biden') 
        elif score < 0:
            verdicts.append('trump')
    return verdicts

def forTrump(polarity):
    verdicts = []
    for score in polarity: 
        if score > 0:
            verdicts.append('trump') 
        elif score < 0:
            verdicts.append('biden')
    return verdicts

biden_df['biden_or_trump'] = forBiden(biden_df['textblob_polarity'])
trump_df['biden_or_trump'] = forTrump(trump_df['textblob_polarity'])

In [7]:
def addHastags(tweets):
    hastags = []
    for t in tweets:
        hastags.append(' '.join(re.findall(r"#(\w+)", t)))
    return hastags

In [8]:
biden_df['nopunct_tweets'] = biden_df['tweet'].str.replace('[^\w\s]','')
trump_df['nopunct_tweets'] = trump_df['tweet'].str.replace('[^\w\s]','')

biden_df['hastags'] = addHastags(biden_df['tweet'])
trump_df['hastags'] = addHastags(trump_df['tweet'])
                         

combined = biden_df.append(trump_df, ignore_index = True)

In [9]:
combined

,index,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,...,country,continent,state,state_code,collected_at,datetime,textblob_polarity,biden_or_trump,nopunct_tweets,hastags
0,5,2020-10-15 00:00:23,1.316529e+18,"""IS THIS WRONG??!!"" Cory Booker's BRILLIANT Fi...",0.0,0.0,Twitter Web App,3.057279e+08,Dose of Dissonance,Dose_Dissonance,...,NaN,NaN,NaN,NaN,2020-10-21 00:00:02.589136415,2020-10-15,0.039583,biden,IS THIS WRONG Cory Bookers BRILLIANT Final Que...,AmyConeyBarrett CoryBooker Barrett Booker Trum...
1,6,2020-10-15 00:00:25,1.316529e+18,"In 2020, #NYPost is being #censorship #CENSORE...",0.0,0.0,Twitter for iPhone,1.994033e+07,Change Illinois | Biden will increase taxes by...,changeillinois,...,United States of America,North America,Illinois,IL,2020-10-21 00:00:03.106963698,2020-10-15,-0.148810,trump,In 2020 NYPost is being censorship CENSORED by...,NYPost censorship CENSORED JoeBiden Trump China
2,7,2020-10-15 00:00:31,1.316529e+18,►► Tell Politicians to STICK IT with this FREE...,0.0,0.0,Freebie-Depot,1.030832e+08,🆓 Freebie Depot,FreebieDepot,...,NaN,NaN,NaN,NaN,2020-10-21 00:00:03.624790981,2020-10-15,0.500000,biden,Tell Politicians to STICK IT with this FREE I...,2020 Biden Deomocrat Election Politician Polit...
3,10,2020-10-15 00:00:44,1.316529e+18,Now Open! Create a FREE U.S. Election Pick'em ...,0.0,0.0,Hootsuite Inc.,2.193205e+07,OfficePools.com,officepools_com,...,NaN,NaN,NaN,NaN,2020-10-21 00:00:05.178272830,2020-10-15,0.404167,biden,Now Open Create a FREE US Election Pickem Pool...,MAGA2020 Trump Biden Politics USElection2020
4,11,2020-10-15 00:00:57,1.316529e+18,"FBI Allegedly Obtained Hunter Biden Computer, ...",0.0,0.0,Twitter for Android,9.607387e+17,RLCompton,NewfoundStudio,...,United States of America,North America,Kentucky,KY,2020-10-21 00:00:05.696100113,2020-10-15,-0.100000,trump,FBI Allegedly Obtained Hunter Biden Computer D...,JoeBiden HunterBiden
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445187,525463,2020-11-03 23:59:53,1.323777e+18,Everyone knows it comes down to PA right? In t...,0.0,0.0,Twitter for iPhone,5.779080e+07,The Chief,TheCHIEF_IN,...,NaN,NaN,NaN,NaN,2020-11-05 12:21:38.916142,2020-11-03,0.065079,trump,Everyone knows it comes down to PA right In th...,election trump biden bitcoin
445188,525464,2020-11-03 23:59:54,1.323777e+18,I’m hoping a few good hours of wind therapy wi...,12.0,0.0,Twitter for iPhone,1.693623e+07,tvthunder,tvwxman3,...,United States of America,North America,Kansas,KS,2020-11-05 12:21:38.780787,2020-11-03,0.266667,trump,Im hoping a few good hours of wind therapy wil...,ElectionNight RedWave Trump2020 Trump TRUMP202...
445189,525466,2020-11-03 23:59:55,1.323777e+18,"Ich hoffe so sehr, dass sich mein komisches Ba...",0.0,0.0,TweetDeck,2.974995e+09,Reaver_Phoenix,Reaver_Phoenix,...,NaN,NaN,NaN,NaN,2020-11-05 12:21:39.014526,2020-11-03,0.227273,trump,Ich hoffe so sehr dass sich mein komisches Bau...,Trump USA Biden USWahl2020 ElectionNight Wahln...
445190,525467,2020-11-03 23:59:57,1.323777e+18,"Dann hoffen wir mal, dass die #USA intelligent...",0.0,0.0,Twitter for iPhone,1.783145e+08,#The_steaming_duck©,Eldudderino,...,Germany,Europe,NaN,NaN,2020-11-05 12:21:39.184971,2020-11-03,1.000000,trump,Dann hoffen wir mal dass die USA intelligent g...,USA Trump Vote


In [10]:
combined['tuple_test'] = combined[['nopunct_tweets', 'biden_or_trump']].apply(tuple, axis=1)
combined['tuple_test_hastag'] = combined[['hastags', 'biden_or_trump']].apply(tuple, axis=1)

In [11]:
#train on non-swing states
train_df = combined[(combined['state'] != 'Pennsylvania') 
                     & (combined['state'] != 'Wisconsin')
                     & (combined['state'] != 'Michigan')]

test_df = combined[(combined['state'] == 'Pennsylvania') 
                     | (combined['state'] == 'Wisconsin')
                     | (combined['state'] == 'Michigan')] 
print("Total number of tweets for the three swing (train) states: ", len(train_df))
print("Total number of tweets for the NON-swing (test) states: ", len(test_df))

Total number of tweets for the three swing (train) states:  436164
Total number of tweets for the NON-swing (test) states:  9028


In [12]:
import random
test = random.sample(list(test_df['tuple_test_hastag']), 200)

In [13]:
from nltk.tokenize import word_tokenize # or use some other tokenizer
import random
train = random.sample(list(train_df['tuple_test_hastag']), 1000)
all_words = set(word.lower() for passage in train for word in word_tokenize(passage[0]))
t = [({word: (word in word_tokenize(x[0])) for word in all_words}, x[1]) for x in train]

In [14]:
import nltk
classifier = nltk.NaiveBayesClassifier.train(t)
classifier.show_most_informative_features()

Most Informative Features
               democrats = True            trump : biden  =      2.8 : 1.0
                election = True            trump : biden  =      2.8 : 1.0
                 america = True            biden : trump  =      2.5 : 1.0
                     usa = True            biden : trump  =      2.5 : 1.0
               biden2020 = True            trump : biden  =      2.2 : 1.0
                    maga = True            trump : biden  =      2.1 : 1.0
             coronavirus = True            trump : biden  =      2.0 : 1.0
                equality = True            biden : trump  =      1.8 : 1.0
                   black = True            biden : trump  =      1.8 : 1.0
                    news = True            biden : trump  =      1.8 : 1.0


In [15]:
test_sentence = "voter fraud"
test_sent_features = {word: (word in word_tokenize(test_sentence.lower())) for word in all_words}
classifier.classify(test_sent_features)

'biden'

In [16]:
test_t = [({word: (word in word_tokenize(x[0])) for word in all_words}, x[1]) for x in test]
print("accuracy percent:",(nltk.classify.accuracy(classifier, test_t)*100))

accuracy percent: 60.5
